<a href="https://colab.research.google.com/github/TSGreenwood/tsgreenwood.github.io/blob/google_cloud/map_bike_lanes_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Map bike lanes using Folium**

In [1]:
# Set the current project in Google Cloud SDK to 'taragreenwoodpersonalwebsite'

!gcloud config set project 'taragreenwoodpersonalwebsite'

Updated property [core/project].


In [2]:
# Connect notebook to my Drive account

from google.colab import auth
auth.authenticate_user()

In [3]:
# Most frequently used libraries

import geopandas as gpd
import folium
from folium import plugins

In [4]:
# Import my mapping functions from the mapping_functions.py file located in my repository into Colab notebook

!wget https://raw.githubusercontent.com/TSGreenwood/tsgreenwood.github.io/google_cloud/kingcountytransitmapping/utils/mapping_functions.py

--2024-03-25 00:13:54--  https://raw.githubusercontent.com/TSGreenwood/tsgreenwood.github.io/google_cloud/kingcountytransitmapping/utils/mapping_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2624 (2.6K) [text/plain]
Saving to: ‘mapping_functions.py’

mapping_functions.p 100%[===================>]   2.56K  --.-KB/s    in 0s      

2024-03-25 00:13:54 (27.0 MB/s) - ‘mapping_functions.py’ saved [2624/2624]



In [5]:
import mapping_functions

In [7]:
# Work directly with GeoJSON dataset stored in Google Cloud bucket

import json
from google.cloud import storage

In [8]:
# Access the GeoJSON File from the Bucket

# Initialize a client
storage_client = storage.Client()

# Define your bucket name and file name
bucket_name = 'kingcountypublictransit'
blob_name = 'data/bike_lanes.geojson'

# Create a bucket object and a blob (file) object
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)

In [9]:
# Ddownload the GeoJSON file directly into memory and load it as a JSON object. This approach avoids saving the file locally

# Download the file's content into memory
geojson_as_string = blob.download_as_string()

# Convert the string to a JSON object
geojson_data = json.loads(geojson_as_string)

Now that the dataset is loaded, we'll do some mapping.

In [11]:
# Initialize basemap

m1 = mapping_functions.create_kc_map()

NameError: name 'folium' is not defined